<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# Growth plugin

**Tags:** #naaschatplugin #naas #naas_driver #chat #plugin #ai

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Description:** This notebook creates a Naas Chat plugin with commands.

## Input

### Import libraries

In [1]:
import naas
from naas_drivers import naas_chat_plugin, gsheet
import naas_data_product
from IPython.display import Markdown
from datetime import date, timedelta
import os
import pandas as pd

✅ utils file '/home/ftp/abi/utils/data.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/llm.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_chat_plugin.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_lab.ipynb' successfully loaded.


### Setup variables
**Mandatory**
- `entity_dir`: This variable represents the entity directory.
- `input_dir`: Input directory to retrieve file from.
- `file_growth`: Name of the file to be retrieved.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `name`: The name of the plugin.
- `model`: This variable holds the openai model.
- `temperature`: This variable holds the temperate value.
- `description`: This variable holds a string that describes the plugin.
- `avatar`: This variable holds a URL to an image to be displayed in your plugin as avatar.
- `model_dir`: This variable holds a relative path to the directory where input notebooks with commands will be stored.

**Optional**
- `output_dir`: This variable holds a relative path to the directory where output files will be stored. 
- `output_path`: This variable holds the full path of the plugin.
- `entity_name`: This variable holds the entity name.

In [2]:
# Inputs
entity_dir = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "entity_dir")
input_dir = os.path.join(entity_dir, "growth-engine", date.today().isoformat())
file_growth = "growth"
spreadsheet_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "abi_spreadsheet") or ""
sheet_name = "GROWTH"
name = "🚀 Growth Assistant"
model = "gpt-4-1106-preview"
temperature = 0.5
description = "Track your growth performance, understand your audience with cohorts analysis and meet your new marketing qualified leads."
avatar = ""
model_dir = os.path.join(naas_data_product.ROOT_PATH, "models", "growth-engine")

# Outputs
output_dir = os.path.join(entity_dir, "plugins")
os.makedirs(output_dir, exist_ok=True) # Create dirs
output_path = os.path.join(output_dir, f"{name.lower().replace(' ', '_')}.json")
entity_name = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "entity_name") or ""

## Model

### Get assets

In [3]:
image_url = get_image_asset(input_dir, "growth_trend.png")
print("- Image URL:", image_url)

image_html = get_image_asset(input_dir, "growth_trend.html")
print("- Image HTML:", image_html)

- Image URL: https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/asset/64f08a826b807c3c18aea1e8db60be704fef46ef5b47b520afbe920289ee.png
- Image HTML: https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/asset/413c4d1814f77c538d2f2c70781991839566c680ad7abee421c3557fabc3


### Get data to be sent to prompt

#### Get content published

In [4]:
# Get content published
content_dir = input_dir.replace("growth-engine", "content-engine")
data_content = pload(content_dir, "content")
if data_content is None:
    data_content = gsheet.connect(spreadsheet_url).get(sheet_name="CONTENT")
if isinstance(data_content, pd.DataFrame):
    df_content = data_content.copy()
    
if len(df_content) > 0:
    df_content = df_content[(df_content["ENTITY"] == entity_name) & (df_content["SCENARIO"].isin([TW, LW]))]
    df_content = df_content[["TITLE", "CONTENT_URL", "DATE"]]

# Transform df to string
post_data = df_content.to_string()

# Count tokens
post_tokens = naas_chat_plugin.num_tokens_from_string(post_data)
print(f"- Post tokens: {post_tokens}")

- Post tokens: 757


#### Get leads generated

In [5]:
# Init
prompt_data = []
total_leads = 0
new_leads = 0

# Load data from pickle
data = pload(input_dir, file_growth)    
if data is None:
    data = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
if isinstance(data, pd.DataFrame):
    df = data.copy()

# Filter on this week and last week
if len(df) > 0:
    df = df[(df["ENTITY"] == entity_name) & (df["SCENARIO"].isin([TW, LW]))]

    # Get data
    total_leads = len(df)
    new_leads = len(df[(df["COHORT"] == "NEW")])

    # Filter to not exceed context windowns
    prompt_data = []
    limit = 0.45
    for row in df.itertuples():
        fullname = row.FULLNAME
        profile_url = row.PROFILE_URL
        occupation = row.OCCUPATION
        last_interaction_date = row.LAST_INTERACTION_DATE
        interactions = row.INTERACTIONS
        string_message = f"{fullname} - '{occupation}' ({profile_url}) interactions: {interactions}"
        prompt_data.append(string_message)
        prompt_tokens, max_tokens = naas_chat_plugin.check_tokens(
            prompt=" ,".join(prompt_data),
            model=model,
            limit=limit,
        )
        if prompt_tokens + post_tokens > max_tokens * limit:
            break
            
print()
print(f"- Leads: {total_leads}")
print(f"- New Leads: {new_leads}")
print(f"- Growth data: {len(prompt_data)}")

✅ System prompt tokens count OK: 323 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 585 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 843 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 1072 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 1273 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 1542 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 1760 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 1979 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 2206 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 2356 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 2513 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 2690 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 2833 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 2971 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 3109 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 3271 (limit: 45% -> 57600)
✅ System prompt tokens count OK: 3435 (limi

### Create system prompt

In [6]:
system_prompt = f"""
Act as a Growth assistant who has access to a list of interactions from content that enable the user to get marketing qualified contacts.
Your role is to manage and optimize the list of people who interacted on the content, ensuring to extract only the most qualified contacts to feed the sales representative.
This week '{TW}' the '{entity_name}' generated {total_leads} leads from them {new_leads} are new thanks to its following posts: {post_data}. 
Here is the list with the highest interaction score: {" ,".join(prompt_data)}.
Please remember the user that you don't have access to all the list due to the limit of the context window.
The first message should be about presenting yourself and analyze briefly '{entity_name}' new interactions with a maximum of 3 bullet points.
Then, you will display the image inside the markdown of the chat about the leads evolution over the weeks: [![Leads Evolution]({image_url})({image_html})].
You must ALWAYS show the image in the first message.
Be casual, but professional. Wait for the first answer from the user, and then start with the first high-level analysis. 
"""

### Create commands

In [7]:
commands = create_command(model_dir)

👌 Well done! Your Notebook has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Notebook as API" feature, just replace .add by .delete
👌 Well done! Your Notebook has been sent to production.



...

PS: to remove the "Notebook as API" feature, just replace .add by .delete
👌 Well done! Your Notebook has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Notebook as API" feature, just replace .add by .delete
[
    {
        "name": "HubSpot_Connect",
        "avatar": "https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/HubSpot.png",
        "action": {
            "request_type": "POST",
            "url": "https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/notebook/4c9eece13a41e6df74605815e8269de0c1f76c2391c2ea7650d10e685d4c",
            "payload": {
                "hs_access_token": {
                    "type": "str",
                    "description": "This variable stores an access token used for accessing the HubSpot API. This value will be stored under the secret 'HS_ACCESS_TOKEN'.",
                    "default": "None"
                }
            }
        },
        "description": "This command integrates your HubSpot access token into the Naas Lab secret manager."
    },
    {
        "name": "LinkedIn_Connect",
        "avatar": "https://raw.githubusercontent

## Output

### Create Naas Chat plugin
This function will generate the plugin in JSON format and also verify if your prompt adheres to the recommended limit, which is set at 20% of the maximum tokens allowed by the model. Then, it will save your plugin in your local environment.

In [8]:
# Create plugin
plugin_file_path = naas_chat_plugin.create_plugin(
    name=name,
    prompt=system_prompt,
    model=model,
    temperature=temperature,
    output_path=output_path,
    description=description,
    avatar=avatar,
    commands=commands
)

⚠️ Be careful, your system prompt looks too big. Tokens: 57971 (limit recommended: 20% -> 25600)
💾 Plugin successfully saved. You can use it in your Naas Chat with: /home/ftp/abi/outputs/jeremy_ravenel/plugins/🚀_growth_assistant.json


### Create asset
This asset can be utilized by using the command `/use` in your Naas Chat or by simply clicking on the link provided in the last cell output.

In [9]:
plugin_url = naas.asset.add(plugin_file_path, params={"inline": True})
print(plugin_url)

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete
https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/asset/4b5b95586486b6a9c588380bdf549361f17f596b3df29c3569a8e115035a


### Create new chat
NB: You don't need to click on 'Create New Chat' everytime you update your system prompt, you can use the command `/refresh`.

In [10]:
Markdown(f"[Create New Chat](https://naas.ai/chat/use?plugin_url={plugin_url})")

[Create New Chat](https://naas.ai/chat/use?plugin_url=https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/asset/4b5b95586486b6a9c588380bdf549361f17f596b3df29c3569a8e115035a)